In [25]:
%env LC_ALL=C
from cgitb import text
import spacy
import os
import csv
import pandas as pd
import numpy as np
from tqdm import tqdm
from spacy.tokens import DocBin
from multiprocessing import Process, freeze_support, set_start_method

env: LC_ALL=C


In [26]:
def load_word_vectors(model_name, word_vectors):
    import spacy
    import subprocess
    import sys
    subprocess.run([sys.executable,
                    "-m",
                    "spacy",
                    "init",
                    "vectors",
                    "en",
                    word_vectors,
                    model_name,
                        ]
                    )
    print (f"New spaCy model created with word vectors. File: {model_name}")

In [ ]:
spacy.prefer_gpu()
load_word_vectors("./models/model-best", "trainset/word_vecs.txt")
nlp = spacy.load("./models/model-best")
nlp.add_pipe("ner")
nlp.to_disk("./models/model-best")

dataframe = pd.read_csv('./dataset/Train_Tagged_Titles.tsv', sep='\t', on_bad_lines='skip', quoting=csv.QUOTE_NONE, encoding='utf8')

In [28]:
listing_titles = [(dataframe["Title"][i], i) for i in range(0, len(dataframe["Title"]))][5000:30000]

In [29]:
input_titles = [(dataframe["Title"][i], i) for i in range(0, len(dataframe["Title"]))][-100:]

In [ ]:
result = []

with tqdm(total = 25000) as pbar:
    buffer = ""
    for doc, i in nlp.pipe(input_titles, as_tuples=True, batch_size=10000):
        for ent in doc.ents:
                
                #have to extract text from listing_titles[i]
                #find the similar text then delete it
                #have to add punctuation back if possible
                
                result.append(str(i + 1) + '\t' + ent.label_ + '\t' + ent.text)
        pbar.update(1)
        
#output data
os.makedirs("./output", exist_ok=True)
output = open("output/output.tsv", "w", encoding="utf-8")  # write mode
output.write('\n'.join(result))
output.close()